# R_14 BulkMove to Partitioned CDB

In [1]:
import json
import subprocess
import copy

In [3]:
with open('../data/curated/Twitter_Cleaned/twitter_cdb_bulk.json', 'r') as f:
    twitter_cdb_bulk = json.load(f)

In [4]:
# change the _id to be partition tolerant
new_twitter_cdb_bulk = {}
out = []
for docs in twitter_cdb_bulk['docs']:
    tmp = copy.deepcopy(docs)
    tmp['_id'] = f"{docs['electorate']}:{docs['_id']}"

    out.append(tmp)

new_twitter_cdb_bulk['docs'] = out

In [7]:
# Upload in bulk to couchdb
for i in range(1280000//10000, len(new_twitter_cdb_bulk['docs'])//10000):
    out = {'docs': new_twitter_cdb_bulk['docs'][i*10000:(i+1)*10000]}
    json_data = json.dumps(out).encode('utf-8')
    subprocess.run(['curl', '-X', 'POST', f'http://172.26.133.251:5984/tweet_database__electorate/_bulk_docs', '--header', 'Content-Type: application/json', '--data-binary', '@-', '-u', 'group9_admin:group9_H1'], input=json_data)

out = {'docs': new_twitter_cdb_bulk['docs'][(i+1)*10000:len(new_twitter_cdb_bulk['docs'])]}
json_data = json.dumps(out).encode('utf-8')
subprocess.run(['curl', '-X', 'POST', f'http://172.26.133.251:5984/tweet_database__electorate/_bulk_docs', '--header', 'Content-Type: application/json', '--data-binary', '@-', '-u', 'group9_admin:group9_H1'], input=json_data)
